# <center>CNN by hand<center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem" data-toc-modified-id="Problem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem</a></span></li><li><span><a href="#Forward-propagation" data-toc-modified-id="Forward-propagation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Forward propagation</a></span><ul class="toc-item"><li><span><a href="#Convolution-Layer" data-toc-modified-id="Convolution-Layer-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Convolution Layer</a></span></li><li><span><a href="#Max-Pooling-Layer" data-toc-modified-id="Max-Pooling-Layer-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Max Pooling Layer</a></span></li><li><span><a href="#Fully-Connected-Layers" data-toc-modified-id="Fully-Connected-Layers-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Fully Connected Layers</a></span></li></ul></li><li><span><a href="#Backpropagation" data-toc-modified-id="Backpropagation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Backpropagation</a></span><ul class="toc-item"><li><span><a href="#Fully-Connected-Layers" data-toc-modified-id="Fully-Connected-Layers-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Fully Connected Layers</a></span></li><li><span><a href="#Pooling-Layers" data-toc-modified-id="Pooling-Layers-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Pooling Layers</a></span></li><li><span><a href="#Convolution-Layers" data-toc-modified-id="Convolution-Layers-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Convolution Layers</a></span></li></ul></li></ul></div>

## Problem 

![title](problem.png)


1. Input size (3,1,1) 
2. One Convolution layer with filter (2,1), stride = 1, pad = 0, depth = 3
3. One max pooling layer wiht filter (2,1) stride = 1, pad = 0
4. 1 Full connection layer with 1 neuron

- Output size of convolution layer = $(3 - 2)/1 + 1 = 2 $  x $(1 - 1)/1 + 1 = 1$ => $(2,1,3)$
- Number of parameter = $2*1*1*3$ = $6$

- Ouput size of max pooling layer = $(2 - 2)/1 + 1 = 1$ x $(1 - 1)/1 + 1 = 1 $ => ($1,1,3)$
- Number of parameter = 0

- FC use 3 parameters

- all biases = 0

## Forward propagation 

In [13]:
import numpy as np

In [23]:
X = np.array([1, 0.4, 0.7])
X = X.reshape(3,1,1)
X

array([[[1. ]],

       [[0.4]],

       [[0.7]]])

### Convolution Layer

Weights of conv layer: take from w1j and i to w3j and i (6 in total)

In [70]:
W1 = np.array([ [[0.2, 0.3,  -0.1]],
                [[0.1, -0.1, 0.2]]
              ])

In [71]:
W1.shape

(2, 1, 3)

In [72]:
X.shape

(3, 1, 1)

In [91]:
Out_conv = np.zeros((2,1,3))

# 1st filter
Out_conv[0,0,0] = np.sum(W1[:,:,0]*X[:2,0,0])
Out_conv[1,0,0] = np.sum(W1[:,:,0]*X[1:3,0,0])

# 2nd filter
Out_conv[0,0,1] = np.sum(W1[:,:,1]*X[:2,0,0])
Out_conv[1,0,1] = np.sum(W1[:,:,1]*X[1:3,0,0])


# 3rd filter
Out_conv[0,0,2] = np.sum(W1[:,:,2]*X[:2,0,0])
Out_conv[1,0,2] = np.sum(W1[:,:,2]*X[1:3,0,0])

In [93]:
Out_conv.shape

(2, 1, 3)

In [92]:
Out_conv

array([[[0.42, 0.28, 0.14]],

       [[0.33, 0.22, 0.11]]])

**=> 6 parameters and output size is (2,1,3)**

### Max Pooling Layer 

In [99]:
Out_pool = np.zeros((1,1,3))

# 1st filter
Out_pool[0,0,0] = np.max(Out_conv[:,:,0])

# 2nd filter
Out_pool[0,0,1] = np.max(Out_conv[:,:,1])

# 3rd filter
Out_pool[0,0,2] = np.max(Out_conv[:,:,2])


In [101]:
Out_pool.shape

(1, 1, 3)

In [100]:
Out_pool

array([[[0.42, 0.28, 0.14]]])

**=> 0 parameter and output size is (1,1,3)**

### Fully Connected Layers

Requirement is 3 parameters, but now only 2 parameters remains (wjk and wik) so another parameter is added (0.6)

- ativation is sigmoid

In [108]:
W3 = np.array([0.1, 0.5, 0.6])
W3 = W3.reshape(1,3)

In [109]:
W3.shape

(1, 3)

In [112]:
Out_pool = Out_pool.reshape(3,1)
Out_pool

array([[0.42],
       [0.28],
       [0.14]])

In [115]:
Z = np.dot(W3,Out_pool)
float(Z)

0.26599999999999996

In [116]:
def sigFun(num):
    return 1/(1 + np.exp(-num))

In [118]:
Output = sigFun(Z)
float(Output)

0.5661106484969565

**=> Result is 0.5661106484969565**

## Backpropagation 

### Fully Connected Layers 

**Define tk = 0.8**

In [119]:
tk = 0.8

In [120]:
dW3 = np.dot(-(tk - Output)*Output*(1 - Output),Out_pool.T)
dW3

array([[-0.02412904, -0.01608603, -0.00804301]])

### Pooling Layers 

Find the input (max) affect to the gradients

In [123]:
dOut_conv = np.zeros(Out_conv.shape)
Out_conv.shape

(2, 1, 3)

In [124]:
def maskPool(x):
    return (x == np.max(x))

In [130]:
#filter 1
dOut_conv[:,:,0] = maskPool(Out_conv[:,:,0])*dW3[0][0] 

#filter 1
dOut_conv[:,:,1] = maskPool(Out_conv[:,:,0])*dW3[0][1] 

#filter 1
dOut_conv[:,:,2] = maskPool(Out_conv[:,:,0])*dW3[0][2] 

In [131]:
dOut_conv

array([[[-0.02412904, -0.01608603, -0.00804301]],

       [[-0.        , -0.        , -0.        ]]])

### Convolution Layers 

I don't know how to calculate gradient of this layer.